# Задача 1 "Задача с именами"

In [201]:
import pandas as pd
import numpy as np
import re

In [202]:
with open ('names.txt', 'r') as f:
    names = f.read()

##  1.1 Сортировка имен в лексикографическом порядке

In [203]:
names = re.split('[^A-Z]', names)
names = sorted([x for x in names if x != ''])

##  1.2 Алфавитная сумма каждого имени

In [1004]:
sum_list = []
for name in names:
    sum_of_name = 0
    for letter in name:
        sum_of_name += ord(letter) - 64
    sum_list.append(sum_of_name)

assert(len(names) == len(sum_list)) #проверка

##  1.3 Произведение алфавитной суммы на порядковый номер

In [1005]:
prod = []
for ind, sum_of_name in enumerate(sum_list):
    prod.append((ind + 1) * sum_of_name)
    
assert(len(prod) == len(sum_list)) #проверка
assert(sum_list[4] * 5 == prod[4])


##  1.4 Сумма произведений алфавитной суммы на порядковые номера

In [229]:
ans = sum(prod)
print(ans)

871853874


# Задача 2 "Задача с ip адресами"

In [970]:
col_names = ['host', 'ip', 'page']
ip_df = pd.read_csv('hits.txt', delimiter='\t', names=col_names)
top_ip = ip_df.groupby('ip').aggregate('count').sort_values('host', ascending=False).reset_index().ip.head()
print(top_ip)

0    154.157.157.156
1     82.146.232.163
2    226.247.119.128
3      194.78.107.33
4     21.143.243.182
Name: ip, dtype: object


# Задача 3 

In [971]:
def del_whitespaces(df):
    '''функция удаления пробелов в массиве'''
    for col in df.columns:
        if type(df[col][1]) is not str:
            continue
        df[col] = df[col].map(str.strip)
    return df

In [1007]:
#чтение данных
lessons = pd.read_csv('tech_quality\lessons.txt', delimiter='|', parse_dates=[3]).drop([0, 379])
lessons = lessons.rename(columns={'                  id                  ': 'lesson_id', 
                ' event_id ': 'event_id', 
                ' subject ': 'subject',
                '       scheduled_time       ': 'scheduled_time'})
lessons['day'] = lessons.iloc[:, 3].dt.day
lessons['event_id'] = lessons['event_id'].astype(int)
lessons = del_whitespaces(lessons)

participants = pd.read_csv('tech_quality\participants.txt', delimiter='|').drop([0, 744])
participants = participants.rename(columns={' event_id ': 'event_id',
                                            '               user_id                ': 'user_id'})
participants = del_whitespaces(participants)
participants['event_id'] = participants['event_id'].astype(int)


quality = pd.read_csv('tech_quality\quality.txt', delimiter='|').drop([0, 366])
quality = quality.rename(columns={'              lesson_id               ': 'lesson_id',
                                  ' tech_quality ': 'tech_quality'})
quality = del_whitespaces(quality)
quality['tech_quality'] = pd.to_numeric(quality['tech_quality'], errors='coerce')


users = pd.read_csv('tech_quality\\users.txt', delimiter = '|').drop([0, 744])
users = users.rename(columns={'                  id                  ': 'user_id', 
                       ' role  ': 'role'})
users = del_whitespaces(users)


In [1008]:
quality_df = quality.groupby('lesson_id').aggregate('count').merge(quality.groupby('lesson_id').aggregate('sum'), on='lesson_id')
quality_df.head()

,tech_quality_x,tech_quality_y
lesson_id,,
00fc6685-f53a-49bb-b960-5e0042fd3852,2,10.0
01f717d8-baaf-427d-a927-08c79713ae79,2,7.0
024d415a-4543-4e73-88c6-f4609e1bf1ed,2,9.0
03f4f4a8-c60c-41d7-a1cb-17b2d36ee183,1,5.0
056c3d91-dce7-4071-91d3-1de816ee63a3,1,4.0


In [1009]:
tutors = users[users['role'] == 'tutor'].user_id
tutors_df = participants[participants['user_id'].isin(tutors)]
tutors = tutors_df.drop_duplicates()

In [1010]:
lessons = lessons[lessons['subject'] == 'phys']
lessons = lessons.merge(tutors_df, how='left', on='event_id').drop_duplicates()
lessons = lessons.merge(quality_df, how='left', on='lesson_id').rename(columns={'tech_quality_x': 'count_of_marks',
                                                                                'tech_quality_y': 'sum_of_marks'})
lessons = lessons.drop(['lesson_id', 'event_id', 'subject', 'scheduled_time'], axis=1).dropna()
lessons = lessons.groupby(['day', 'user_id'])['count_of_marks', 'sum_of_marks'].aggregate('sum')
lessons.head()

count_of_marks  sum_of_marks
day user_id                                                           
11  2fa2ab62-f1b0-4036-872f-bcfd9a8686ff             1.0           5.0
    30a19496-bdaf-461c-abbc-2709ae520201             1.0           5.0
    43efce48-94b2-4412-857f-223d45969008             3.0          15.0
    603b8641-c6f6-4d89-ac89-88e50d45aa0d             5.0          24.0
    8fe03f08-8581-430c-a590-9888ab36deb3             7.0          31.0

In [1011]:
lessons['mean_mark'] = round(lessons['sum_of_marks'] / lessons['count_of_marks'], 2)
lessons = lessons.drop(['sum_of_marks', 'count_of_marks'], axis=1).reset_index()

In [1012]:
lessons = lessons.groupby('day').agg('min').reset_index()
lessons

,day,user_id,mean_mark
0,11,2fa2ab62-f1b0-4036-872f-bcfd9a8686ff,4.43
1,12,2fa2ab62-f1b0-4036-872f-bcfd9a8686ff,4.89
2,13,2fa2ab62-f1b0-4036-872f-bcfd9a8686ff,5.00
3,14,2fa2ab62-f1b0-4036-872f-bcfd9a8686ff,4.00
4,15,603b8641-c6f6-4d89-ac89-88e50d45aa0d,5.00
5,16,2fa2ab62-f1b0-4036-872f-bcfd9a8686ff,4.00
6,17,30a19496-bdaf-461c-abbc-2709ae520201,4.50
7,18,1c39e78f-6ded-4b2e-83d6-036ca34ecfdc,4.25
8,19,43efce48-94b2-4412-857f-223d45969008,4.50
9,20,2fa2ab62-f1b0-4036-872f-bcfd9a8686ff,4.50
